# Genus 2 case


So for a curve of genus $g=2$ we get
```{math}
:label:
    f(x,y) = -y^2 + x^5 + \lambda_2 x^4 + \lambda_4 x^3+ \lambda_6 x^2+ \lambda_8 x+ \lambda_{10},
```  
Let's load the appropriate SageMath library and define the curve $\mathscr{C}$

In [1]:
from sage.schemes.riemann_surfaces.riemann_surface import RiemannSurface

In [2]:
# Defines the lambda coefficients
lambda2 = 12.0
lambda4 = 2.0
lambda6  = 0.9
lambda8  = 21.0
lambda10  = 10.0

```{important}
The coefficients above must be real-floating-point numbers 
```

Since the current Sage library only works well on the field of rational numbers, we have to approximate all function coefficients by these numbers.


In [3]:
# Rational approximation
l2 = lambda2.nearby_rational(max_error=1e-10)
l4 = lambda4.nearby_rational(max_error=1e-10)
l6 = lambda6.nearby_rational(max_error=1e-10)
l8 = lambda8.nearby_rational(max_error=1e-10)
l10 = lambda10.nearby_rational(max_error=1e-10)

Next, we need to define the variables $x$ and $y$ in the ring of polynomials over the rational numbers.

In [4]:
R.<x, y> = PolynomialRing(QQ, 2)

```{note}
The order of variables can be important depending on how you write your polynomial. Make sure you use the same order in your polynomial expression.
```

In [5]:
# Defining the polynomial f
f = -y^2 + x^5 + l2*x^4 + l4*x^3 + l6*x^2 + l8*x + l10

## Riemann surface

````{prf:definition}
:label: riemann_sur

A Riemann surface $X$ is a connected two-dimensional topological manifold with a complex-analytic structure on it. 
````

The SageMath RiemannSurface library provides a function to generate the appropriate Riemann surface, on which we will continue our work.

In [6]:
S = RiemannSurface(f, prec=100)

## Branch points

We define a function to compute and display all branch points $e_i$ based on the given coefficients $\lambda_{2i}$. Branch points correspond to the zeros of the polynomial defined by $f(x, y) = 0$. Specifically, the polynomial is given by:
```{math}
:label:
    y^2 = (x - e_1)(x - e_2)(x - e_3)(x - e_4)(x - e_5).
```
Here, the branch points $e_i$ are the roots of the polynomial on the right-hand side, which describe the structure of the Riemann surface associated with $f(x, y) = 0$.

In [7]:
def find_branch_points(ll2, ll4, ll6, ll8, ll10):
    # We define a ring of polynomials over the field of complex numbers
    CC_poly.<x> = PolynomialRing(CC)
    
    # We create a polynomial
    pol = x^5 + ll2*x^4 + ll4*x^3 + ll6*x^2 + ll8*x + ll10
    
    # We find roots
    roots = pol.roots(multiplicities=False)
    
    # We add a point at infinity if the degree of the polynomial is odd
    if pol.degree() % 2 == 1:
        roots.append(infinity)
    
    return roots

```{note}
The `multiplicities=False` parameter in the `roots()`  method in Sage has the following meaning:
- When `multiplicities=False`  (default): The method returns only the roots of the polynomial, without information about their multiplicities. The result is a list of unique root values.
- When `multiplicities=True`: The method returns pairs (root, multiplicity) for each root. The result is a list of tuples, where each tuple contains the root and its multiplicity.
```

In [8]:
# Example of use:

branch_points = find_branch_points(l2, l4, l6, l8, l10)
print("Branch points:", *branch_points, sep='\n')

Branch points:
-11.8251161409669
-1.05264146874943
-0.512317960119317
0.695037784917798 - 1.04164546422316*I
0.695037784917798 + 1.04164546422316*I
+Infinity


## First and Second Kind Periods

To construct periodic functions, we first define a canonical basis for the space of holomorphic differentials, \( \{du_i \mid i = 1, \ldots, g\} \), and the associated meromorphic differentials, \( \{dr_i \mid i = 1, \ldots, g\} \), on the Riemann surface as follows:
```{math}
:label:
    du_{2i-1} := \frac{x^{g-i} dx}{\partial_y f(x)},
```
```{math}
:label:
    dr_{2i-1} := \frac{\mathcal{R}_{2i-1}(x) dx}{\partial_y f(x)}, 
```
where
```{math}
:label:
    \mathcal{R}_{2i-1}(x)=\sum_{k=1}^{2i-1}k\lambda_{4i-2k-2}x^{g-i+k}.
```
For \( g = 2 \), these expressions can be written in vector form as:
```{math}
:label:
    du= \begin{pmatrix} 
        x\\
        1
    \end{pmatrix} \frac{dx}{-2\sqrt{f(x)}},
```
```{math}
:label:
    dr= \begin{pmatrix} 
        \mathcal{R}_{1}(x)\\
        \mathcal{R}_{3}(x)
    \end{pmatrix} \frac{dx}{-2\sqrt{f(x)}} = 
        \begin{pmatrix} 
            x^2\\
            3x^3 + 2\lambda_2 x^2 + \lambda_4 x
        \end{pmatrix} \frac{dx}{-2\sqrt{f(x)}},
```

The holomorphic basis defined above can be compared to the output of the`cohomology_basis()` function. In SageMath,`S.cohomology_basis()` generates a list of holomorphic differentials, typically represented as polynomials $g(x)$ corresponding to the differentials:
```{math}
:label:
    \omega = g(x,y)\frac{dx}{\partial f / \partial y}
```
where  $f(x, y) = 0$  defines the curve.


In [9]:
S.cohomology_basis()

[1, x]

```{note}
The order of elements clearly differs from the convention adopted in {cite}`bernatska_computation_2024`. In this notebook, we adopt Julia Bernatska’s convention, as it provides a consistent framework for our computations.
```

###  First Kind Periods    

In [10]:
# holomorphic differentials base
holbais=[x,x^0]

To compute the period matrices of the first kind, we evaluate the following integrals along the canonical homology cycles $\{\mathfrak{a}_i, \mathfrak{b}_i\}_{i=1}^g$:
```{math}
:label:
    \omega = (\omega_{ij})= \left( \int_{\mathfrak{a}_j}du_i \right), \quad \omega' = (\omega'_{ij})= \left( \int_{\mathfrak{b}_j}du_i \right). 
```
Here, $\omega$ and $\omega'$ are the period matrices corresponding to the $\mathfrak{a}$ and $\mathfrak{b}$ cycles, respectively.
  
The SageMath function `matrix_of_integral_values(differentials, integration_method='heuristic')` can be used to compute the path integrals of the given differentials along the homology basis. The result is a matrix, where each row corresponds to a differential.
  
If the Riemann surface is given by the equation $f(x,y)=0$, the differentials are encoded by:
```{math}
:label:
    g(x,y)\frac{dx}{(df/dy)}.
```
`Input:`
- `differentials` – a list of polynomials.
- `integration_method` – (default: 'heuristic'). String specifying the integration method to use. The options are 'heuristic' and 'rigorous'.

`Output:`  
A matrix, one row per differential, containing the values of the path integrals along the homology basis of the Riemann surface.

In [11]:
MofInt1=S.matrix_of_integral_values(holbais)
# Let's display the matrix in a shortened form so that it will be easy to see its structure
print(MofInt1.n(digits=5))

[     0.13760 - 0.66033*I  -4.9304e-32 - 0.91722*I  -3.9443e-31 + 0.40344*I       1.1746 + 0.25689*I]
[    -0.62448 + 0.28220*I -5.5467e-32 + 0.038862*I  -4.9304e-32 - 0.52554*I      0.14607 + 0.24334*I]


The structure of the returned matrix
```{math}
:label:
    MofInt1 = 
        \begin{pmatrix}
            \omega_{1,1} & \omega_{1,3} & \omega'_{1,1} & \omega'_{1,3} \\
            \omega_{3,1} & \omega_{3,3} & \omega'_{3,1} & \omega'_{3,3}
        \end{pmatrix}
        =
        \begin{array}{|c|c|c|c|c|}
            \hline
            & \mathfrak{a}_1 & \mathfrak{a}_3 & \mathfrak{b}_1 & \mathfrak{b}_3 \\
            \hline
            x & \omega_{1,1} = \int_{\mathfrak{a}_1} du_1 & \omega_{1,3} = \int_{\mathfrak{a}_3} du_1 & \omega'_{1,1} = \int_{\mathfrak{b}_1} du_1 & \omega'_{1,3} = \int_{\mathfrak{b}_3} du_1 \\
            \hline
            1 & \omega_{3,1} = \int_{\mathfrak{a}_1} du_3 & \omega_{3,3} = \int_{\mathfrak{a}_3} du_3 & \omega'_{3,1} = \int_{\mathfrak{b}_1} du_3 & \omega'_{3,3} = \int_{\mathfrak{b}_3} du_3 \\
            \hline
        \end{array}
```

We can compare this with the results of the built-in Sage function: 'period_matrix()', which, for the adopted notational convention, will return a period matrix in the form
```{math}
:label:
    pM= \begin{pmatrix}
                \omega_{3,1} & \omega_{3,3} & \omega'_{3,1} & \omega'_{3,3} \\
                \omega_{1,1} & \omega_{1,3} & \omega'_{1,1} & \omega'_{3,1}
        \end{pmatrix}  
```

In [12]:
pM=S.period_matrix()
print(pM.n(digits=5))

[   -0.62448 + 0.28220*I 4.3141e-32 + 0.038862*I -4.9304e-32 - 0.52554*I     0.14607 + 0.24334*I]
[    0.13760 - 0.66033*I  2.2187e-31 - 0.91722*I -5.9165e-31 + 0.40344*I      1.1746 + 0.25689*I]


Before going any further, lat's define a function that will display the matrices in a rounded form so that we can compare them more easily.

In [13]:
def format_complex(z, digits=5, threshold=1e-10):
    real = float(z.real())
    imag = float(z.imag())
    
    # We round very small values to zero
    if abs(real) < threshold:
        real = 0
        if abs(imag) < threshold:
            return "0"
        # We format the result  
        return f"{imag:.{digits}f}*I"
    
    if abs(imag) < threshold:
        # We format the result
        return f"{real:.{digits}f}"

    sign = "+" if imag > 0 else "-"
    return f"{real:.{digits}f} {sign} {abs(imag):.{digits}f}*I"


def ApproxM(matrix, digits=5, threshold=1e-10):
    rows, cols = matrix.nrows(), matrix.ncols()
    
    for i in range(rows):
        formatted_row = [format_complex(matrix[i,j], digits, threshold) for j in range(cols)]
        print("\t".join(formatted_row))

In [14]:
ApproxM(pM)

-0.62448 + 0.28220*I	0.03886*I	-0.52554*I	0.14607 + 0.24334*I
0.13760 - 0.66033*I	-0.91722*I	0.40344*I	1.17459 + 0.25689*I


In [15]:
ApproxM(MofInt1)

0.13760 - 0.66033*I	-0.91722*I	0.40344*I	1.17459 + 0.25689*I
-0.62448 + 0.28220*I	0.03886*I	-0.52554*I	0.14607 + 0.24334*I


In what follows we use the function `matrix_of_integral_values()` instead of `period_matrix()` because it allows us to calculate periodic matrices of the second kind.

In [16]:
# Extract the omega-periods (first two columns)
omega = MofInt1[:, 0:2]

# Extract the omega'-periods (last two columns)
omegaP = MofInt1[:, 2:4]

ApproxM(omega)
print()
ApproxM(omegaP)

0.13760 - 0.66033*I	-0.91722*I
-0.62448 + 0.28220*I	0.03886*I

0.40344*I	1.17459 + 0.25689*I
-0.52554*I	0.14607 + 0.24334*I


Now we calculate the matrix
```{math}
:label:
    \tau=\omega^{-1}\omega'
```
which belongs to the Siegel upper half-space. Hence it should satisfy two conditions:
- Symmetry:
```{math}
:label:
    \tau^T = \tau
```
- Positive definiteness of the imaginary part
```{math}
:label:
    Im(\tau)>0
```

In [17]:
tau= omega.inverse() * omegaP

# Displaying the result
print(tau.n(digits=5))

[-0.28894 + 0.70313*I -0.12636 - 0.46286*I]
[-0.12636 - 0.46286*I  -0.25854 + 1.6328*I]


In [18]:
# Test of the symmetry
print(tau-tau.transpose().n(digits=5))

[0.00000 0.00000]
[0.00000 0.00000]


In [19]:
# Test of positivity
# Calculating the complex part of the tau matrix
tauImag = tau.apply_map(lambda x: x.imag())

# Calculate the eigenvalues
eigenvalues = tauImag.eigenvalues()

# Checking if all eigenvalues are positive
all_positive = all(e > 0 for e in eigenvalues)

# Displaying the result
eigenvalues, all_positive

([1.8239307353738361091933820951, 0.51198440541283078719843599741], True)

### Second Kind Periods

To calculate the second kind period matrices, we need to calculate the following integrals along the canonical homology cycles $\{ \mathfrak{a}_i, \mathfrak{b}_i\}_{i=1}^g$
```{math}
:label:
    \eta = (\eta_{ij})= \left( \int_{\mathfrak{a}_j}du_i \right), \quad \eta' = (\eta'_{ij})= \left( \int_{\mathfrak{b}_j}du_i \right). 
```

In [20]:
# meromorphic differentials base
merbais=[x^2, 3*x^3 + 2*l2*x^2 + l4*x]

In [21]:
MofInt2=S.matrix_of_integral_values(merbais)
# Let's display the matrix in a shortened form so that it will be easy to see its structure
ApproxM(MofInt2)

0.15574 + 0.20627*I	0.08372*I	-0.32882*I	-7.07532 + 0.12255*I
4.74867 + 3.09497*I	-0.04832*I	-6.23827*I	-2.96268 + 3.14329*I


In [22]:
# Extract the omega-periods (first two columns)
eta = MofInt2[:, 0:2]

# Extract the omega'-periods (last two columns)
etaP = MofInt2[:, 2:4]

ApproxM(eta)
print()
ApproxM(etaP)

0.15574 + 0.20627*I	0.08372*I
4.74867 + 3.09497*I	-0.04832*I

-0.32882*I	-7.07532 + 0.12255*I
-6.23827*I	-2.96268 + 3.14329*I


We can compute $\kappa$, given by
```{math}
:label:
    \kappa=\eta\; \omega^{-1}
```

In [23]:
omega_inv=Matrix(omega).inverse()
kappa = eta*omega_inv
ApproxM(kappa)

-0.09763 - 0.01261*I	-0.14978 - 0.29754*I
-0.14978 - 0.29754*I	-4.77835 - 7.02263*I


## Legendre relation

Next, we perform another test. The unnormalised period matrices of the first kind, $\omega$ and $\omega'$, and the second kind, $\eta$ and $\eta'$, should satisfy the Legendre relation:
 ```{math}
:label:
    \Omega^T J \Omega = 2\pi i J
```
where

```{math}
:label:
    \Omega=
    \begin{pmatrix}
        \omega && \omega'\\
        \eta && \eta'\\
    \end{pmatrix}, \quad 
    J=        
    \begin{pmatrix}
        0 && -1_g\\
        1_g && 0\\
    \end{pmatrix}
```

In [24]:
# Omega matrix
Omega = block_matrix([
    [omega, omegaP],
    [eta, etaP]
])

# Converting lists to matrices
zeroM = Matrix([[0.0, 0.0], [0.0, 0.0]])
mOneg = Matrix([[-1.0, 0.0], [0.0, -1.0]])
Oneg = Matrix([[1.0, 0.0], [0.0, 1.0]])

# J matrix
J = block_matrix([
    [zeroM, mOneg],
    [Oneg, zeroM]
])    


print("Omega Matrix:")
print(Omega.n(digits=5))
print()
print("J Matrix:")
print(J.n(digits=5))

Omega Matrix:
[     0.13760 - 0.66033*I  -4.9304e-32 - 0.91722*I| -3.9443e-31 + 0.40344*I       1.1746 + 0.25689*I]
[    -0.62448 + 0.28220*I -5.5467e-32 + 0.038862*I| -4.9304e-32 - 0.52554*I      0.14607 + 0.24334*I]
[-------------------------------------------------+-------------------------------------------------]
[     0.15574 + 0.20627*I  7.8886e-31 + 0.083724*I| -8.3816e-31 - 0.32882*I      -7.0753 + 0.12255*I]
[       4.7487 + 3.0950*I  1.1360e-28 - 0.048319*I|   2.2088e-29 - 6.2383*I       -2.9627 + 3.1433*I]

J Matrix:
[0.00000 0.00000|-1.0000 0.00000]
[0.00000 0.00000|0.00000 -1.0000]
[---------------+---------------]
[ 1.0000 0.00000|0.00000 0.00000]
[0.00000  1.0000|0.00000 0.00000]


In [25]:
import numpy as np

pi = np.pi
left=Omega.transpose()*J*Omega
right = 2*pi*I*J
result = left - right
ApproxM(result)

0	0	0	0
0	0	0	0
0	0	0	0
0	0	0	0


## Characteristics of branch points

````{prf:definition}
:label: Char_df

A characteristic is a $2\times g$ matrix $[\varepsilon] = (\mathbf{\varepsilon}', \mathbf{\varepsilon})^T$ with real values within the interval $[0,2)$. Every point $u$ in the funcamental domain $\mathrm{Jac}(\mathscr{C})$ can be represented by its characteristic $[\varepsilon]$:
```{math}
:label:
    u = \frac{1}{2} \omega \mathbf{\varepsilon} + \frac{1}{2}\omega' \mathbf{\varepsilon}'.
```
    In the hyperelliptic case, the Abel images of branch points and any combination of branch points are described by characteristics with 1 or 0, which are called <i>half-integer</i> characteristics.
````

The half-integer characteristics are odd whenever 
```{math}
    \varepsilon^T \varepsilon'=0 \pmod{2},
```
and even when 
```{math}
    \varepsilon^T \varepsilon'= 1\pmod{2},
```
For an Abelian image of a branch points
```{math}
    \mathcal{A}_{i,0} = \int^{\infty}_{e_i} du
```
we can write
```{math}
    \mathcal{A}_{i,0} = \frac{1}{2} \omega \mathbf{\varepsilon}_i + \frac{1}{2}\omega' \mathbf{\varepsilon}'_i.
```
Hence the characteristics of $\mathcal{A}_{i,0}$ is given by
```{math}
    [\mathcal{A}_{i,0}] = 
        \begin{pmatrix}
            \mathbf{\epsilon'}_i^T\\
            \mathbf{\epsilon}_i^T
        \end{pmatrix} =
        \begin{pmatrix}
            \epsilon'_{i,1} & \ldots & \epsilon'_{i,g} \\
            \epsilon_{i,1} & \ldots &\epsilon_{i,g}
        \end{pmatrix}.
```
For $g=2$ 
```{math}
    [\mathcal{A}_{i,0}] = 
        \begin{pmatrix}
            \epsilon'_{i,1} & \epsilon'_{i,2} \\
            \epsilon_{i,1} & \epsilon_{i,2}
        \end{pmatrix}
```

In [26]:
# We define the genus variable
genus = S.genus  


eChars = [[[0 for k in range(genus)], [1 for k in range(genus)]], 
          [[0 for k in range(genus)] for i in range(2)]]

# Do-like loop in Julia's Mathematica notebook
for l in range(genus):
    # let's note the indexing, which must be adjusted by
    eChars.insert(0, 
        [[(eChars[0][0][k] + kronecker_delta(k+1, genus - l) + 
           kronecker_delta(k+1, genus - l + 1)) % 2 for k in range(genus)],
         [(eChars[0][1][k] + 0) % 2 for k in range(genus)]]
    )

    eChars.insert(0,
        [[(eChars[0][0][k] + 0) % 2 for k in range(genus)],
         [(eChars[0][1][k] + kronecker_delta(k+1, genus - l)) % 2 for k in range(genus)]]
    )

# We display matrices
seen_matrices = []
for i in range(len(eChars)):
    current_matrix = matrix(eChars[i])
    if current_matrix not in seen_matrices:
        seen_matrices.append(current_matrix)
        print(current_matrix)
        print()

[1 0]
[0 0]

[1 0]
[1 0]

[0 1]
[1 0]

[0 1]
[1 1]

[0 0]
[1 1]

[0 0]
[0 0]



```{note}
This result can be compared with the result at work {cite}`enolski_periods_2007` (p. 14, example 5.1)
```  

There are 2 odd characteristic among them, their sum is also odd. The corresponding vector -vector of Riemann constants -is denoted by $\mathbf{K}$ and 
```{math}
    [\mathbf{K}] = [\mathcal{A}_{2,0}+\mathcal{A}_{4,0}]
```

In [27]:
# We sum the eChars elements with indices 2*i +1 (because python counts from 0) and take Mod 2
KCh = sum(matrix(eChars[2 * i+1]) for i in range(genus)) % 2

print(KCh)

[1 1]
[0 1]


## Theta function

````{prf:definition}
:label: theta_df

A Riemann <i>theta function</i> $\theta(v;\tau)$ defined in terms of normalized coordinates $v$ and normalized period matrix $\tau$, canonicaly is given by
```{math}
:label:
    \theta(\mathbf{v};\tau) = \sum_{n\in\mathbb{Z}^g} e^{i\pi \mathbf{n}^T \tau \mathbf{n} + 2i \pi \mathbf{n}^T \mathbf{v}}.
```
A theta function with characteristic $[\varepsilon ]$ is defined by
```{math}
:label:
    \theta[\varepsilon](\mathbf{v};\tau) = 
    \theta 
    \begin{bmatrix}
            \varepsilon'_1 & \ldots & \varepsilon'_g\\
            \varepsilon_1 & \ldots & \varepsilon_g\\
    \end{bmatrix}
    (\mathbf{v};\tau)=
    \sum_{n\in\mathbb{Z}^g} e^{i\pi\{ (\mathbf{n}+ \frac{1}{2}\mathbf{\varepsilon}'^T ) \tau (\mathbf{n}+ \frac{1}{2}\mathbf{\varepsilon}') +2 (\mathbf{v}+\frac{1}{2}\mathbf{\varepsilon})^T (\mathbf{n}+\frac{1}{2}\mathbf{\varepsilon}') \} }.
```
````

````{important}
In the literature, it is common to use a convention with slightly different characteristics:
```{math}
:label:
    (\varepsilon_i, \varepsilon'_j) \to 2 (\varepsilon_i, \varepsilon'_j)
```
in consequence
```{math}
:label:
    \theta[\varepsilon](\mathbf{v};\tau) = 
    \theta 
    \begin{bmatrix}
        \varepsilon'_1 & \ldots & \varepsilon'_g\\
        \varepsilon_1 & \ldots & \varepsilon_g\\
    \end{bmatrix}
    (\mathbf{v};\tau)=
    \sum_{n\in\mathbb{Z}^g} e^{i\pi\{ (\mathbf{n}+ \mathbf{\varepsilon}'^T ) \tau (\mathbf{n}+ \mathbf{\varepsilon}') +2 (\mathbf{v}+\mathbf{\varepsilon})^T (\mathbf{n}+\mathbf{\varepsilon}') \} }.
```
````

In [28]:
def Theta( v, ttau, NAcc):
    # NAcc is responsible for the number of elements in the sum, i.e. the precision of the result. 
    # Experimentally, a good approximation is obtained for NAcc>4, but of course this can be increased as needed.
    total_sum = 0
    # epsilon_m is the list [epsilon 1, epsilon 2] where epsilon1 and epsilon2 are vectors
    
    # We iterate over two indices from -NAcc to NAcc
    for n1 in range(-NAcc, NAcc):
        for n2 in range(-NAcc, NAcc):
            # We create vector n
            n = vector([n1, n2])
                    
            # The first component of the sum
            term1 = I * pi * n * (ttau * n)
                    
            # The second component of the sum
            term2 = 2 * I * pi * n * v
                    
            # We add the exp from these components to the total
            total_sum += exp(term1 + term2)
    
    return total_sum

In [29]:
def ThetaCh(epsilon_m, v, ttau, NAcc):
    # NAcc is responsible for the number of elements in the sum, i.e. the precision of the result. 
    # Experimentally, a good approximation is obtained for NAcc>4, but of course this can be increased as needed.
    total_sum = 0
    # epsilon_m is the list [epsilon 1, epsilon 2] where epsilon1 and epsilon2 are vectors
    epsilon1 = epsilon_m[0]
    epsilon2 = epsilon_m[1]
    
    # We iterate over two indices from -NAcc to NAcc
    for n1 in range(-NAcc, NAcc):
        for n2 in range(-NAcc, NAcc):
            # We create vector n
            n = vector([n1, n2])
                    
            # The first component of the sum
            term1 = I * pi * (n + 1/2 * vector(epsilon1)) * (ttau * (n + 1/2 * vector(epsilon1)))
                    
            # The second component of the sum
            term2 = 2 * I * pi * (n + 1/2 * vector(epsilon1)) * (v + 1/2 * vector(epsilon2))
                    
            # We add the exp from these components to the total
            total_sum += exp(term1 + term2)
    
    return total_sum

````{note}
In Wolfram Mathematica exists a corresponding function under the name `SiegelTheta[$\nu_1$,$\nu_2$]($\Omega,s$)`. The relation between our variables and those in Mathematica are as follows
- $\Omega=\tau$
- $s=v$
- $\nu_1 = \frac{1}{2} epsilon1$ 
- $\nu_2 = \frac{1}{2} epsilon2$  
  
To test this, you can check the following code in Mathematica
```{code-block}
:caption: Mathematica Code      
            SiegelTheta[{{2, 4}, {4, 2}}, IdentityMatrix[2] I, {1, 2}] // N
```
and compare it with below one in SageMath
```{code-block}
:caption: SageMath Code              
    # Defining sample data
    epsilon_m = [vector([4, 8]), vector([8, 4])]
    v = vector([1, 2])
    tau = Matrix([[I, 0], [0, I]])
    NAcc = 5
    # Function call
    result = ThetaCh(epsilon_m, v, tau, NAcc)
    print(result)
```
  
Additionally, one can define the $\theta$ function in Mathematica in a similar way with the following code:
```{code-block}
:caption: Mathematica Code    
    ThetaCh[\[Epsilon]m_, v_, \[Tau]_, NAcc_] :=
    Sum[
     Exp[
      I Pi ({Subscript[n, 1], Subscript[n, 2]} + 1/2 \[Epsilon]m[[1]]) . (\[Tau] . ({Subscript[n, 1], Subscript[n, 2]} + 1/2 \[Epsilon]m[[1]])) + 2 I Pi ({Subscript[n, 1], Subscript[n, 2]} + 1/2 \[Epsilon]m[[1]]) . (v + 1/2 \[Epsilon]m[[2]])
      ],
     {Subscript[n, 2], -NAcc, NAcc}, {Subscript[n, 1], -NAcc, NAcc}
    ]
```
  
And check the result of the above test with: 
```{code-block}
:caption: Mathematica Code   
    ThetaCh[2 {{2, 4}, {4, 2}}, {1, 2}, IdentityMatrix[2] I, 5] // N
```
````

### Tests

Test of the formula ({cite}`bernatska_computation_2024`, p.4, eq. 7)
```{math}
:label:
    \theta[\varepsilon](\mathbf{v};\tau) = 
            e^{i\pi\left(\frac{1}{2} \mathbf{\varepsilon}'^T\right) \tau \left(\frac{1}{2} \mathbf{\varepsilon}'\right) + 2i\pi \left( \mathbf{v} + \frac{1}{2} \mathbf{\varepsilon}\right)^T \left(\frac{1}{2} \mathbf{\varepsilon}'\right)} \theta\left(\mathbf{v} + \frac{1}{2}\mathbf{\varepsilon} + \tau\left(\frac{1}{2}\mathbf{\varepsilon}'\right);\tau \right).   
```

where a characteristic is a a $2\times g$ matrix $[\varepsilon] =(\mathbf{\varepsilon}',\mathbf{\varepsilon})^T $    


In [30]:
# Define the genus
g = 2  # For genus 2

Acc=20

# Define the period matrix tau
### tau = Matrix(CC, [[I, 0.5], [0.5, I]])
tau = omega.inverse() * omegaP

# Define the vector v
v = vector(CC, [0.1, 0.2])

# Define the characteristic
eps_prime = [1, 0]
eps = [0, 1]
N = 2  # Level of the characteristic
char = Matrix([eps_prime,eps])

# Compute theta with characteristic at v
theta_char = ThetaCh(char, v, tau, Acc)

# Compute half of the characteristic vectors
eps_vec = vector(CC, eps)
eps_prime_vec = vector(CC, eps_prime)
eps_half = 0.5 * eps_vec
eps_prime_half = 0.5 * eps_prime_vec

# Compute the shifted vector v_shifted
v_shifted = v + eps_half + tau * eps_prime_half

# Compute the terms for the exponential factor
term1 = (eps_prime_half) * tau * (eps_prime_half)
term2 = (v + eps_half) * (eps_prime_half)

# Compute the exponential factor
exp_factor = exp(I * pi * term1 + 2 * I * pi * term2)

# Compute theta at the shifted point without characteristic
theta_standard = Theta(v_shifted, tau, Acc)


# Compute the RHS of the relation
RHS = exp_factor * theta_standard

# Compute the difference
difference = theta_char - RHS

# Print the results
print("Theta with characteristic:\n", theta_char)
print()
print("Exponential factor:\n", exp_factor)
print()
print("Theta at shifted point:\n", theta_standard)
print()
print("RHS:\n", RHS)
print()
print("Difference:\n", difference)

# Check if the difference is within an acceptable tolerance
tolerance = 1e-12  # Adjust based on the precision
if abs(difference) < tolerance:
    print("The relation is verified within the given tolerance.")
else:
    print("The relation is not satisfied within the given tolerance.")

Theta with characteristic:
 1.0600858517074845 - 0.2501023731212625*I

Exponential factor:
 0.5734712442681018 + 0.050148801540618036*I

Theta at shifted point:
 1.7966652347912195 - 0.5932345951363659*I

RHS:
 1.0600858517074845 - 0.2501023731212626*I

Difference:
 1.1102230246251565e-16*I
The relation is verified within the given tolerance.


## $\sigma$-Functions

````{prf:definition}
:label: sigma_df

<em> Sigma function </em> (Kleinian sigma) is a modular invariant entire function on $\mathrm{Jac}(\mathscr{C})$. It is definef by a relation with the theta function:
```{math}
:label:
    \sigma(\mathbf{u}) = C \tilde{\sigma}(\mathbf{u})
```
where
```{math}
:label:
    \tilde{\sigma}(\mathbf{u})= e^{-\frac{1}{2}\mathbf{u}^T \kappa \mathbf{u}} \theta[ K ] (\omega^{-1} \mathbf{u}, \omega^{-1} \omega'),
```
```{math}
:label:
    C= \sqrt{\frac{\pi^g}{\det{\omega}}} \left( \prod_{1\leq i<j \leq 2g+1} (e_i - e_j) \right)^{-1/4},
```
and $[K]$ denotes the characteristics of the vector of Riemann constants. (The expression for $C$ comes from {cite}`enolski_inversion_2012`, p.9, Eq. II.41)
````

In [31]:
# We define variables
var('U1 U3')

# We define the accuracy of theta function
Acc=20


# sigma
def Tsigma(U1, U3):
    e = exp(-(1/2)*(vector([U1, U3])*kappa*vector([U1, U3])))
    theta = ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)
    return e*theta

# C constant
det_omega = omega.determinant()
g = 2
#Branch points
BP = find_branch_points(l2, l4, l6, l8, l10)
# Calculating the product of branch point differences
prod = 1
for i in range(len(BP)):
    for j in range(i+1, len(BP)):
        if BP[i] != infinity and BP[j] != infinity:
            prod *= (BP[i] - BP[j])
C = sqrt(pi**g / det_omega) * prod**(-1/4)


def sigma(U1, U3):
    return C*Tsigma(U1, U3)

### Bolza formula

````{prf:definition}
:label: sigma_app1_df

From {cite}`bernatska_computation_2024`(Sec. 2.4), we have the Bolza formula:
```{math}
    e_\iota = - \frac{\partial_{u_3}\theta[{\iota}](\omega^{-1}u)}{\partial_{u_1}\theta[{\iota}](\omega^{-1}u)}|_{u=0},
```
where $[{\iota}]$ denotes the characteristic corresponding to a branch point $e{\iota}$. It can also be defined in an equivalent way:
```{math}
    e_\iota = - \frac{\sigma_3(u_\iota)}{\sigma_1(u_\iota)},
```
where $u_\iota=\frac{1}{2}\omega \epsilon_\iota + \frac{1}{2}\omega' \epsilon'_\iota$. 
````    

In [32]:
epsp1 = vector(eChars[0][0])
eps1 = vector(eChars[0][1])

epsp2 = vector(eChars[1][0])
eps2 = vector(eChars[1][1])

epsp3 = vector(eChars[2][0])
eps3 = vector(eChars[2][1])

epsp4 = vector(eChars[3][0])
eps4 = vector(eChars[3][1])

epsp5 = vector(eChars[4][0])
eps5 = vector(eChars[4][1])

In [33]:
up1 = (1/2)*omega*eps1 + (1/2)*omegaP*epsp1
up2 = (1/2)*omega*eps2 + (1/2)*omegaP*epsp2
up3 = (1/2)*omega*eps3 + (1/2)*omegaP*epsp3
up4 = (1/2)*omega*eps4 + (1/2)*omegaP*epsp4
up5 = (1/2)*omega*eps5 + (1/2)*omegaP*epsp5

In [34]:
# ei
def e(u1_val, u3_val):
    U1, U2 = var('U1 U3')
    sigma_expr = Tsigma(U1, U3)
    sigma1 = diff(sigma_expr, U1)
    sigma3 = diff(sigma_expr, U3)
    symbolic_expr = -sigma3/sigma1
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()

In [35]:
print("Branch points from Bolza formula:", 
      e(up1[0],up1[1]), 
      e(up2[0],up2[1]), 
      e(up3[0],up3[1]), 
      e(up4[0],up4[1]), 
      e(up5[0],up5[1]), sep='\n')

Branch points from Bolza formula:
-1.05264146874943 - 3.16880859405019e-16*I
0.695037784917798 + 1.04164546422316*I
-11.8251161409668 - 5.32963482422400e-14*I
2.69103715492428e15 - 6.17663031281984e15*I
0.695037784917798 - 1.04164546422316*I


In [36]:
print("Branch points:", *branch_points, sep='\n')

Branch points:
-11.8251161409669
-1.05264146874943
-0.512317960119317
0.695037784917798 - 1.04164546422316*I
0.695037784917798 + 1.04164546422316*I
+Infinity


Ok, let's see how the $\sigma_i$ functions behave at the points $up_4$

In [37]:
def check_sigma_values(u1_val, u3_val):
    U1, U3 = var('U1 U3')
    sigma_expr = Tsigma(U1, U3)
    sigma1 = diff(sigma_expr, U1)
    sigma3 = diff(sigma_expr, U3)
    
    print("sigma1:", sigma1.subs({U1: u1_val, U3: u3_val}).n())
    print("sigma3:", sigma3.subs({U1: u1_val, U3: u3_val}).n())

check_sigma_values(up4[0], up4[1])

sigma1: 1.55163978531505e-15 + 3.73708216078305e-15*I
sigma3: -27.2580952691324 - 0.472721613117682*I


````{prf:observation}
$\sigma_1$=0 so $up4$ is a special divisor
````
````{prf:observation}
The order of the elements should be the same.
````

## $\wp$-Functions

````{prf:definition}
:label: wp_df

Multiply periodic Klein-Weierstrass $\wp$-functions are defined by
```{math}
:label:
    \wp_{ij}(\mathbf{u}):= - \frac{\partial^2\log{\sigma(\mathbf{u})}}{\partial u_i \partial u_j}, \quad \wp_{ijk}:=- \frac{\partial^3\log{\sigma(\mathbf{u})}}{\partial u_i \partial u_j \partial u_k},
```
where $\sigma(u)$ is called <i>sigma function</i>. This can be also written by 
```{math}
:label:
    \wp_{ij} = \frac{\sigma_i(\mathbf{u})\sigma_j(\mathbf{u}) - \sigma(\mathbf{u})\sigma_{ij}(\mathbf{u})}{\sigma^2(\mathbf{u})} = \frac{\tilde{\sigma}_i(\mathbf{u})\tilde{\sigma}_j(\mathbf{u}) - \tilde{\sigma}(\mathbf{u})\tilde{\sigma}_{ij}(\mathbf{u})}{\tilde{\sigma}^2(\mathbf{u})},
```
where $\sigma_i(\mathbf{u})$ denotes the derivative of the sigma function with respect to the $i$-th component of $\mathbf{u}$. 
  
It can be shown that the above definitions can be written in the form
```{math}
:label:
    \wp_{ij}:=\kappa_{ij} - \frac{\partial^2}{\partial u_i \partial u_j}\log{\theta[K](\omega^{-1}u;\tau)}, \quad \wp_{ijk}:=- \frac{\partial^3}{\partial u_i \partial u_j \partial u_k}\log{\theta[K](\omega^{-1}u;\tau)}.
```
````        

In [38]:
#definition with thetas
# We define variables
var('U1 U3')

# We define the accuracy of theta function
Acc=20


# WeierstrassP11
def WeierstrassP11(u1_val, u3_val):
    symbolic_expr = kappa[0, 0] - diff(log(ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)), U1, 2)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()

# WeierstrassP13
def WeierstrassP13(u1_val, u3_val):
    symbolic_expr = kappa[0, 1] - diff(log(ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)),  U1, U3)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()

# WeierstrassP33
def WeierstrassP33(u1_val, u3_val):
    symbolic_expr = kappa[1, 1] - diff(log(ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)), U3, 2)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()

# WeierstrassP1111
def WeierstrassP1111(u1_val, u3_val):
    symbolic_expr = - diff(log(ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)), U1, 4)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()

# WeierstrassP3333
def WeierstrassP3333(u1_val, u3_val):
    symbolic_expr =  - diff(log(ThetaCh(KCh, omega_inv * vector([U1, U3]), tau, Acc)), U3, 4)
    return symbolic_expr.subs({U1: u1_val, U3: u3_val}).n()


### Tests

As a first test of these functions, we can check if they satisfy the key property  
```{math}
:label:
    \wp_{ij}(\mathbf{u}+2\omega \mathbf{n} + 2\omega'\mathbf{n}') = \wp_{ij}(\mathbf{u}),
```
where
```{math}
:label:
    \mathbf{n} = 
        \begin{pmatrix}
            n_1\\
            n_2
        \end{pmatrix}, \quad
    \mathbf{n}' = 
        \begin{pmatrix}
            n'_1\\
            n'_2
        \end{pmatrix} \in \mathbb{Z}^2.
```

In [39]:
ntest = vector([1, 2])
nPtest = vector([-3, -5])

wn = omega*ntest
wPn= omegaP*nPtest

print("Theta based definitions")
print()
print("Test P11:")
print(WeierstrassP11(2.0, 3.0) - WeierstrassP11(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))
print()
print("Test P13:")
print(WeierstrassP13(2.0, 3.0) - WeierstrassP13(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))
print()
print("Test P33:")
print(WeierstrassP33(2.0, 3.0) - WeierstrassP33(2.0 + 2*wn[0] + 2*wPn[0], 3.0 + 2*wn[1] + 2*wPn[1]))
print()

Theta based definitions

Test P11:


-2.20040874410188e-10 - 3.79429820895893e-12*I

Test P13:


-3.93356458516791e-11 - 8.13997758086771e-11*I

Test P33:


3.04964942188235e-11 - 4.54747350886464e-11*I



### Basic relations

#### I.
````{prf:definition}

The following formula cames from {cite}`bernatska_reality_2024` (Eq. 44)  - this is KdV equation
```{math}
:label:
    \wp_{1111} = 6\wp^2_{11} +  4\lambda_2 \wp_{11} + 4 \wp_{13} + 2 \lambda_4 
```

````

In [40]:
u1 = 0.212131
u3 = -2.231

RHS = 6 * WeierstrassP11(u1,u3)^2 + 4 * l2 * WeierstrassP11(u1,u3) + 4 * WeierstrassP13(u1,u3) + 2*l4
RHS

634.524660261415 - 7.34873484042567e-12*I

In [41]:
LHS = WeierstrassP1111(u1,u3)
LHS

634.524660261410 - 4.54747350886464e-12*I

In [42]:
LHS - RHS

-5.57065504835919e-12 + 2.80126133156103e-12*I

Let's check this for another value of vector $u$

In [43]:
u1 = -1.98213
u3 = 3.76213

RHS = 6 * WeierstrassP11(u1,u3)^2 + 4 * l2 * WeierstrassP11(u1,u3) + 4 * WeierstrassP13(u1,u3) + 2*l4
LHS = WeierstrassP1111(u1,u3)
LHS - RHS

7.02016222930979e-12 + 9.76584775600947e-12*I

````{prf:observation}
Works great :)
````

#### II.

````{prf:definition}
:label: 
Based on {cite}`enolski_inversion_2012` (5.10-13) let's take
```{math}
:label:
    \Omega_{ij} = \frac{1}{2} \omega (\epsilon_i + \epsilon_j) +  \frac{1}{2} \omega' (\epsilon'_i + \epsilon'_j),
```
then
```{math}
:label:
    \wp_{11}(\Omega_{ij}) = e_i + e_j, \quad \wp_{13}(\Omega_{ij}) = -e_i e_j,  \quad i,j=1,\ldots,5, \; i\neq j
```

````

In [44]:
Omega12 = (1/2)*omega*(eps1 + eps2) + (1/2)*omegaP*(epsp1 + epsp2)
Omega13 = (1/2)*omega*(eps1 + eps3) + (1/2)*omegaP*(epsp1 + epsp3)
Omega15 = (1/2)*omega*(eps1 + eps5) + (1/2)*omegaP*(epsp1 + epsp5)
Omega23 = (1/2)*omega*(eps2 + eps3) + (1/2)*omegaP*(epsp2 + epsp3)
Omega25 = (1/2)*omega*(eps2 + eps5) + (1/2)*omegaP*(epsp2 + epsp5)
Omega35 = (1/2)*omega*(eps3 + eps5) + (1/2)*omegaP*(epsp3 + epsp5)

In [45]:
print("P11(Omega12)=",WeierstrassP11(Omega12[0],Omega12[1]))
print("From theory: e1+e2=",e(up1[0],up1[1])+ e(up2[0],up2[1]))
print("From practice: e3+e5=",e(up3[0],up3[1])+ e(up5[0],up5[1]))

P11(Omega12)= -11.1300783560491 - 1.04164546422316*I


From theory: e1+e2= -0.357603683831631 + 1.04164546422316*I


From practice: e3+e5= -11.1300783560490 - 1.04164546422322*I


In [46]:
print("P13(Omega12)=",WeierstrassP13(Omega12[0],Omega12[1]))
print("From theory: -e1*e2=",-e(up1[0],up1[1])* e(up2[0],up2[1]))
print("From practice: -e3*e5=",-e(up3[0],up3[1]) * e(up5[0],up5[1]))

P13(Omega12)= 8.21890252901330 - 12.3175785921502*I


From theory: -e1*e2= 0.731625594752220 + 1.09647921137605*I


From practice: -e3*e5= 8.21890252901334 - 12.3175785921502*I


In [47]:
print("P11(Omega13)=",WeierstrassP11(Omega13[0],Omega13[1]))
print("From theory: e1+e3=",e(up1[0],up1[1])+ e(up3[0],up3[1]))
print("From practice: e2+e5=",e(up5[0],up5[1]) + e(up2[0],up2[1]))

P11(Omega13)= 1.39007556983560 + 9.32587340685131e-15*I


From theory: e1+e3= -12.8777576097163 - 5.36132291016450e-14*I


From practice: e2+e5= 1.39007556983560 - 1.11022302462516e-15*I


In [48]:
print("P11(Omega15)=",WeierstrassP11(Omega15[0],Omega15[1]))
print("From theory: e1+e5=",e(up1[0],up1[1])+ e(up5[0],up5[1]))
print("From practice: e2+e3=",e(up3[0],up3[1]) + e(up2[0],up2[1]))

P11(Omega15)= -11.1300783560491 + 1.04164546422316*I


From theory: e1+e5= -0.357603683831631 - 1.04164546422316*I


From practice: e2+e3= -11.1300783560490 + 1.04164546422311*I


In [49]:
print("Branch points from Bolza formula:", 
      e(up1[0],up1[1]), 
      e(up2[0],up2[1]), 
      e(up3[0],up3[1]), 
      e(up4[0],up4[1]), 
      e(up5[0],up5[1]), sep='\n')

Branch points from Bolza formula:
-1.05264146874943 - 3.16880859405019e-16*I
0.695037784917798 + 1.04164546422316*I
-11.8251161409668 - 5.32963482422400e-14*I
2.69103715492428e15 - 6.17663031281984e15*I
0.695037784917798 - 1.04164546422316*I


In [50]:
print("Branch points:", *branch_points, sep='\n')

Branch points:
-11.8251161409669
-1.05264146874943
-0.512317960119317
0.695037784917798 - 1.04164546422316*I
0.695037784917798 + 1.04164546422316*I
+Infinity


## Jacobi inversion problem on branch points

Let $\mathbf{u}=\mathcal{A}(D)$ be the Abel image of a degree g positive non-special divisor $D$ on the curve. Then $D$ is uniquely defined by the system of equations:
```{math}
:label:
        \mathcal{R}_{2g}(x;\mathbf{u}) = x^g - \sum_{i=1}^g x^{g-1} \wp_{1,2i-1}(\mathbf{u}) =0,
```
```{math}
:label:
        \mathcal{R}_{2g+1}(x,y;\mathbf{u}) = 2y + \sum_{i=1}^{g-i} x^{g-1} \wp_{1,1,2i-1}(\mathbf{u}) =0.
```
So in case of $g=2$:    
```{math}
:label:
        \mathcal{R}_4(x;\mathbf{u})=x^2 - x\wp_{11}(\mathbf{u})- \wp_{13}(\mathbf{u})=0,
```   
```{math}
:label:
        \mathcal{R}_5 (x;\mathbf{u})= 2y + x\wp_{111}(\mathbf{u}) + \wp_{113}(\mathbf{u})=0.
```

````{prf:definition}
:label: divisor_df

Let 
```{math}
        D=\sum_{i=1}^n P_i,
```
be a divisor on a curve $\mathscr{C}$. We assume that $D$ is <b>non-special</b>, that is 
- $ n\geq g$,
- $D$ does not contain pairs of points connected by the hyperelliptic involution.

````

```{note}

On special divisors $\sigma$-function vanishes, and so $\wp$-functions have singularities.

```

As a point $P_i$ we choose
```{math}
:label:
    P_i = (x_i,y_i) = (x_i, y_{+}(x_i)),
```
where $y_{+}(x) = + \sqrt{y^2(x)}$ (there is second option $y_{-}(x) = - \sqrt{y^2(x)}$ ). 

In our case let's take
```{math}
:label:
    P_1 = (x_1, y_1)=(1.5,y_{+}(1.5))
```
```{math}
:label:
    P_2 = (x_2, y_2)=(3.1,y_{+}(3.1))
```

In [51]:
# Definition of y function 
def y(x):
    res = sqrt(x^5 + l2*x^4 + l4*x^3 + l6*x^2 + l8*x + l10)
    return res.n()

To calculate $\mathbf{u}$ I will use the Sage function:  `abel_jacobi()`, where it expects lists of tuples in the format  `(v,P)`, where  `v` is the multiplicity of the point in the divisor (in our case 1 for both points), and  `P` is a tuple  `(x,y)` representing the point on the curve. 
  
The multiplicity `v` (also called valuation) in the context of divisors determines how many times a given point appears in the divisor. Here are some key points:
- For regular points on the curve that are neither singular points nor points at infinity, typically $v = 1$ or $v = -1$ is used.
$v = 1$ means the point appears positively in the divisor (it is "added").
- $v = -1$ means the point appears negatively in the divisor (it is "subtracted").
- If a point appears multiple times, $v$ can be greater than 1 or less than -1.
- For special points (e.g., points at infinity or singular points), v may take other values depending on the local structure of the curve at that point.
  
  
In your case, where you define the divisor as a sum of two regular points: `D = [x1 + x2, y1 + y2]` each of these points appears once positively in the divisor, so for both points`v = 1`. Therefore, divisor in the format suitable for the `abel_jacobi()` function would look like this:  

In [52]:
x1 = 0.256
x2 = 11.721

y1 = y(x1)
y2 = y(x2)

# P_i points
P1 = [ x1, y1 ] 
P2 = [ x2, y2 ]

# Divisor
divisor = [(1, (x1, y1)), (1, (x2, y2))]

```{note}
If one wanted to define a more complex divisor, for example, the difference of two points, it might look like this:  
`divisor = [(1, (x1, y1)), (-1, (x2, y2))]`  
where -1 indicates that the second point is subtracted in the divisor.
```

In [53]:
AJ = S.abel_jacobi(divisor)

print(AJ)

(-0.095227328487916328568064649224 - 0.47103938123276306779408678278*I, -0.94610300133296732955826058561 - 0.36651365565790567383047992719*I)


### Tests

In [54]:
u1=AJ[0]
u3=AJ[1]

print("P11(u):")
print(WeierstrassP11(u1, u3))

print("P13(u):")
print(WeierstrassP13(u1, u3))

print("P33(u):")
print(WeierstrassP33(u1, u3))

P11(u):


6.36829653384861 - 4.59081552648913*I
P13(u):


-9.85938695314932 - 1.97507933943491*I
P33(u):


-23.8259011047194 + 49.1062328287841*I


In [55]:
mat1R4=[[1, x, x^2],[ 1, x1, x1^2], [1, x2, x2^2]]
det1=matrix(mat1R4).determinant()
mat2R4=[[ 1, x1], [1, x2]]
det2=matrix(mat2R4).determinant()
det1/det2

x^2 - 11.9770000000000*x + 3.00057600000000

````{prf:observation}
Ok, this is bad. 
````

## Bibliography

```{bibliography}
:style: unsrt
:filter: docname in docnames
```